In [1]:
import pandas as pd
import numpy as np

### 下载文件到本地

In [5]:
import requests

url1 = 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv'
url2 = 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv'
url3 = 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv'

response1 = requests.get(url1)
with open('state-population.csv', 'wb') as f:
    f.write(response1.content)

response2 = requests.get(url2)
with open('state-areas.csv', 'wb') as f:
    f.write(response2.content)

response3 = requests.get(url3)
with open('state-abbrevs.csv', 'wb') as f:
    f.write(response3.content)

### 读取文件，查看数据集

In [6]:
pop = pd.read_csv('state-population.csv') 
areas = pd.read_csv('state-areas.csv') 
abbrevs = pd.read_csv('state-abbrevs.csv') 
print(pop.head())
print(areas.head())
print(abbrevs.head())

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


### 连接数据集
多对一合并获取人口（pop）DataFrame 中各州名称缩写对应的全称

丢弃重复信息

In [11]:
merged = pd.merge(pop, abbrevs, how='outer', 
                  left_on='state/region', right_on='abbreviation') 
merged = merged.drop('abbreviation', axis=1) # 丢弃重复信息
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


### 全面检查一下数据是否有缺失

In [12]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

### 查看缺失值

In [13]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [14]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

### 快速填充对应的全称

In [15]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico' 
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States' 
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

### 用两个数据集共同的 state 列,将面积数据合并

In [16]:
final = pd.merge(merged, areas, on='state', how='left') 
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


### 再次检查缺失值

In [17]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [18]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

### 去掉缺失值，因为全国的人口密度在此无关紧要

In [19]:
final.dropna(inplace=True) 
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


### 数据准备完成
使用query()函数，选择2000 年的各州人口以及总人口数据

In [20]:
data2010 = final.query("year == 2010 & ages == 'total'") 
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


### 计算人口密度并按序排列
首先对索引进行重置，然后再计算结果

In [21]:
data2010.set_index('state', inplace=True) 
density = data2010['population'] / data2010['area (sq. mi)'] 

density.sort_values(ascending=False, inplace=True) 
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

# 结果结论
美国各州加上华盛顿特区（Washington, DC）、波多黎各在 2010 年的人口密度排序，以万人 / 平方英里为单位

人口密度最高的地区是华盛顿特区的哥伦比亚地区（the District of Columbia）

在各州的人口密度中，新泽西州（New Jersey）是最高的

### 查看人口密度最低的3个州

In [24]:
density.tail(3)

state
Montana    6.736171
Wyoming    5.768079
Alaska     1.087509
dtype: float64